In [223]:
import pandas as pd
import psycopg2

In [224]:
# Function to create database
def create_database():
    
    #Establish connection with default database
    conn = psycopg2.connect(
        host = '127.0.0.1',
        dbname = 'postgres',
        user = 'postgres',
        password = 'root'
    )
    
    conn.set_session( autocommit= True) # automate commit
    cur = conn.cursor()   # get cursor
    
    # Create database
    # cur.execute("DROP DATABASE education") -- if exists
    cur.execute("CREATE DATABASE education")
    
    # Close connection to default database
    cur.close()
    
    # connect to the created database
    conn = psycopg2.connect(
        host = '127.0.0.1',
        dbname = 'education',
        user = 'postgres',
        password = 'root'
        )
    
    cur =conn.cursor()
    
    return cur, conn
    
    

In [225]:
# # function to delete table
# def drop_table(cur,conn):
#     for query in drop_table_queries:
#         cur.execute(query)
#         conn.commit()

In [226]:
# # function to create table
# def create_table(cur,conn):
#     for query in create_table_queries:
#         cur.execute(query)
#         conn.commit()

In [233]:
TeacherStatus =pd.read_csv('Data/teacher-by-status-and-county-for-secondary-csv-1.csv')
TeacherCounty = pd.read_csv ('Data/teachers-per-county-in-kenya-csv-2.csv')

In [234]:
TeacherStatus.head()

,COUNTY,School Type,Employment Body,No. of Teachers,County (centroid),Year
0,Baringo,Public,TSC,1172,"(0.669252, 35.946465)",12/31/2014 12:00:00 AM
1,Bomet,Public,TSC,1379,"(-0.726295, 35.298598)",12/31/2014 12:00:00 AM
2,Bungoma,Public,TSC,2946,"(0.749285, 34.640461)",12/31/2014 12:00:00 AM
3,Busia,Public,TSC,1182,"(0.387444, 34.193631)",12/31/2014 12:00:00 AM
4,Elgeyo Marakwet,Public,TSC,1087,"(0.802219, 35.536563)",12/31/2014 12:00:00 AM


In [235]:
# Rename columns
TeacherStatus.rename({
     "COUNTY":"County",
     "No. of Teachers": "No of Teachers"    
 
},axis = 1 , inplace = True 
)

In [236]:
TeacherStatus['Year'] = pd.to_datetime(TeacherStatus['Year'])
TeacherCounty['Year'] = pd.to_datetime(TeacherCounty['Year'])

In [237]:
TeacherStatus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   County             192 non-null    object        
 1   School Type        192 non-null    object        
 2   Employment Body    192 non-null    object        
 3   No of Teachers     192 non-null    int64         
 4   County (centroid)  188 non-null    object        
 5   Year               192 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 9.1+ KB


In [238]:
# rename columns
TeacherCounty.rename({ "Public": "Employment Body"}, axis =1 , inplace = True)

In [239]:
TeacherCounty.tail()

,County,Employment Body,Total Number,County (Centroid),Year
189,Uasin Gishu,Privat,1620.0,"(0.5260859, 35.3220117)",2014-12-31
190,Vihiga,Privat,561.0,"(0.0762658, 34.722323)",2014-12-31
191,Wajir,Privat,237.0,"(1.8081376, 40.0349856)",2014-12-31
192,West Pokot,Privat,201.0,"(1.7401062, 35.2438468)",2014-12-31
193,Kenya,Privat,75406.0,NaN,2014-12-31


In [240]:
TeacherCounty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   County             194 non-null    object        
 1   Employment Body    194 non-null    object        
 2   Total Number       192 non-null    float64       
 3   County (Centroid)  188 non-null    object        
 4   Year               194 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 7.7+ KB


In [241]:
print("Shape of the TeacherStatus: ",TeacherStatus.shape)
print("Shape of the TeacherCounty: ", TeacherCounty.shape)

Shape of the TeacherStatus:  (192, 6)
Shape of the TeacherCounty:  (194, 5)


In [253]:
TeacherCounty.tail()

,County,Employment Body,Total Number,County (Centroid),Year
189,Uasin Gishu,Privat,1620.0,"(0.5260859, 35.3220117)",2014-12-31
190,Vihiga,Privat,561.0,"(0.0762658, 34.722323)",2014-12-31
191,Wajir,Privat,237.0,"(1.8081376, 40.0349856)",2014-12-31
192,West Pokot,Privat,201.0,"(1.7401062, 35.2438468)",2014-12-31
193,Kenya,Privat,75406.0,NaN,2014-12-31


In [244]:
TeacherStatus.head()

,County,School Type,Employment Body,No of Teachers,County (centroid),Year
0,Baringo,Public,TSC,1172,"(0.669252, 35.946465)",2014-12-31
1,Bomet,Public,TSC,1379,"(-0.726295, 35.298598)",2014-12-31
2,Bungoma,Public,TSC,2946,"(0.749285, 34.640461)",2014-12-31
3,Busia,Public,TSC,1182,"(0.387444, 34.193631)",2014-12-31
4,Elgeyo Marakwet,Public,TSC,1087,"(0.802219, 35.536563)",2014-12-31


In [245]:
TeacherCounty.nunique()

County                49
Employment Body        5
Total Number         189
County (Centroid)     47
Year                   1
dtype: int64

In [247]:
TeacherStatus.nunique()

County                48
School Type            2
Employment Body        3
No of Teachers       141
County (centroid)     47
Year                   1
dtype: int64

### Some data cleaning ...


In [248]:
TeacherStatus.select_dtypes(include =['object']).columns

Index(['County', 'School Type', 'Employment Body', 'County (centroid)'], dtype='object')

In [249]:
# Remove leading trailing spaces from the string data
omit_column = 'County (Centroid)'

for column in TeacherCounty.select_dtypes(include =['object']).columns:
    if column!= omit_column:
        TeacherCounty[column] = TeacherCounty[column].str.strip()

In [250]:
# Remove leading trailing spaces from the string data
omit_column = 'County (Centroid)'

for column in TeacherStatus.select_dtypes(include =['object']).columns:
    if column!= omit_column:
        TeacherStatus[column] = TeacherStatus[column].str.strip()

In [261]:
TeacherStatus['Employment Body'].unique()

array(['TSC', 'SMC', 'Private'], dtype=object)

In [255]:
TeacherCounty['Employment Body'].unique()

array(['TSC', 'Public', 'SMC', 'Total', 'Privat'], dtype=object)

In [260]:
TeacherCounty[TeacherCounty['Employment Body'] == 'Total']

,County,Employment Body,Total Number,County (Centroid),Year
98,Baringo,Total,5750.0,"(0.669252, 35.9464654)",2014-12-31
99,Bomet,Total,6493.0,"(-0.7262946, 35.2985976)",2014-12-31
100,Bungoma,Total,9920.0,"(0.7492852, 34.6404609)",2014-12-31
101,Busia,Total,5637.0,"(0.3874437, 34.1936315)",2014-12-31
102,Elgeyo Marakwet,Total,4229.0,"(0.8022188, 35.5365631)",2014-12-31
103,Embu,Total,4230.0,"(-0.6039744, 37.6265013)",2014-12-31
104,Garissa,Total,1115.0,"(-0.4889103, 40.1992785)",2014-12-31
105,Homa Bay,Total,8189.0,"(-0.5428094, 34.3589969)",2014-12-31
106,Isiolo,Total,933.0,"(1.0104431, 38.5421531)",2014-12-31
107,Kajiado,Total,3633.0,"(-2.1213836, 36.9088422)",2014-12-31


In [262]:
TeacherStatus[TeacherStatus["County"]== 'Kenya']

,County,School Type,Employment Body,No of Teachers,County (centroid),Year
47,Kenya,Public,TSC,72194,NaN,2014-12-31
95,Kenya,Public,SMC,35524,NaN,2014-12-31
143,Kenya,Private,Private,10890,NaN,2014-12-31
191,Kenya,Private,TSC,0,NaN,2014-12-31


Observation: From the table TeacherStatus the sum of all teachers is under the county 'Kenya' so to ensure data inetgrity we create a new dataFrame from this and delete the 'Kenya' rows from the Teacher Status dataframe

In [264]:
total_teachers_df = TeacherStatus.loc[TeacherStatus['County'] == 'Kenya']
total_teachers_df

,County,School Type,Employment Body,No of Teachers,County (centroid),Year
47,Kenya,Public,TSC,72194,NaN,2014-12-31
95,Kenya,Public,SMC,35524,NaN,2014-12-31
143,Kenya,Private,Private,10890,NaN,2014-12-31
191,Kenya,Private,TSC,0,NaN,2014-12-31


In [265]:
# Drop the rows where the County is Kenya from the TeacherStatus DataFrame
TeacherStatus = TeacherStatus.drop(TeacherStatus[TeacherStatus['County'] == 'Kenya'].index)

In [268]:
TeacherStatus['County'].nunique()

47

In [269]:
TeacherStatus['Employment Body'].unique()

array(['TSC', 'SMC', 'Private'], dtype=object)

In [ ]:
TeacherStatus['School Type'].unique()

array(['Public', 'Private'], dtype=object)

### Webscraping county data

In [270]:
# Fetch data for the county and their codes from url
import requests
from bs4 import BeautifulSoup 


url = "https://en.wikipedia.org/wiki/Counties_of_Kenya" # URL containing the list of Kenyan counties

# Fetch the list of counties from the URL
response = requests.get(url)
response.raise_for_status()  # Raise an error for bad responses

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with counties (assumed to be the first table on the page)
table = soup.find('table', {'class': 'wikitable'})

In [271]:
# Extract counties from the table
data = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) >= 3:  # Ensure there are enough columns
        county_data = {
            'Code': cells[0].get_text(strip=True),
            'County': cells[1].get_text(strip=True)
            
        }
        data.append(county_data)

# Convert to DataFrame
counties_data = pd.DataFrame(data)
counties_data.set_index('Code', inplace=True)

In [272]:
# Renaming the counties
counties_data["County"] = counties_data["County"].replace(
    {
        "Elgeyo-Marakwet" : 'Elgeyo Marakwet',
        "Trans-Nzoia": 'Trans Nzoia',
        "Murang'a" : 'Muranga',
        "Taita–Taveta": 'Taita Taveta'
      
        
    }
)

In [273]:
counties_data.head()

,County
Code,
1,Mombasa
2,Kwale
3,Kilifi
4,Tana River
5,Lamu


### Extract Unique Dimension Data

In [274]:
# Extract unique values for dimensions
counties = TeacherStatus['County'].unique()
school_types = TeacherStatus['School Type'].unique()
employment_bodies = TeacherStatus['Employment Body'].unique()

In [275]:
# Prepare unique dataframes for dimension tables
counties_df = pd.DataFrame(counties, columns= ['County'])
school_type_df = pd.DataFrame(school_types, columns=['School Type'])
employment_body_df = pd.DataFrame(employment_bodies, columns=['Employment Body'])

In [276]:
# Extract years
years = TeacherStatus["Year"].unique()
year_df = pd.DataFrame(years, columns=['Year'])

In [277]:
year_df

,Year
0,2014-12-31


In [278]:
TeacherStatus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188 entries, 0 to 190
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   County             188 non-null    object        
 1   School Type        188 non-null    object        
 2   Employment Body    188 non-null    object        
 3   No of Teachers     188 non-null    int64         
 4   County (centroid)  188 non-null    object        
 5   Year               188 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 10.3+ KB


In [280]:
# counties_df["County"] =  counties_data["County"].replace({})

In [279]:
import numpy as np

# Validating the county data
# Step 1: Extract and clean county names
df1_counties = counties_data['County'].str.strip()
df2_counties = counties_df['County'].str.strip()

# Step 2: Convert to numpy arrays for easier comparison
df1_counties = df1_counties.values
df2_counties = df2_counties.values

# Step 3: Find differences between the two lists
missing_in_df2 = np.setdiff1d(df1_counties, df2_counties)  # Present in df1 but missing in df2
extra_in_df2 = np.setdiff1d(df2_counties, df1_counties)    # Present in df2 but missing in df1

print("Counties in df1 but not in df2:", missing_in_df2)
print("Counties in df2 but not in df1:", extra_in_df2)

Counties in df1 but not in df2: []
Counties in df2 but not in df1: []


# From csv to database

In [ ]:
# connect to database
cur, conn = create_database()

#### Creating the dimension tables

In [282]:
try:
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS County (
            County_ID SERIAL PRIMARY KEY,
            County VARCHAR(255) ,
            County_Centroid VARCHAR(255),
            UNIQUE (County, County_Centroid)
            
        );
        
        """
    )
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Error creating County table: {e}")      
    

In [283]:
try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS School_Type (
            School_Type_ID SERIAL PRIMARY KEY,
            School_Type VARCHAR(255) UNIQUE
        );
    """)
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Error creating School_Type table: {e}")


In [284]:
try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Employment_Body (
            Employment_Body_ID SERIAL PRIMARY KEY,
            Employment_Body VARCHAR(255) UNIQUE
        );
    """)
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Error creating Employment_Body table: {e}")

In [285]:
try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Year (
            Year_ID SERIAL PRIMARY KEY,
            Year DATE UNIQUE
        );
    """)
    conn.commit()
except psycopg2.Error as e:
    conn.rollback()
    print(f"Error creating Year table: {e}")

In [307]:
# Fact table

fact_table = """

CREATE TABLE IF NOT EXISTS Fact_Table (
    Fact_ID SERIAL PRIMARY KEY,
    County_ID INT REFERENCES County(County_ID),
    School_Type_ID INT REFERENCES School_Type(School_Type_ID),
    Employment_Body_ID INT REFERENCES Employment_Body(Employment_Body_ID),
    Year_ID INT REFERENCES Year(Year_ID),
    No_of_Teachers INT
);


"""

In [308]:
try:
    cur.execute(fact_table)
    conn.commit()
except Exception as e:
    conn.rollback()  
    print(f"Error creating Fact table: {e}")

### Insert data into the tables

In [311]:
# List of queries
county_table_insert = (
    """
    INSERT INTO County (County, County_Centroid )
    VALUES (%s, %s)
    ON CONFLICT (County, County_Centroid) DO NOTHING;
    """
)

school_type_insert = (
    """
    INSERT INTO School_Type (School_Type) VALUES (%s)
    ON CONFLICT (School_Type) DO NOTHING;

    """
) 


employment_body_insert = (
    """
    INSERT INTO Employment_Body (Employment_Body )
    VALUES (%s)
    ON CONFLICT (Employment_Body) DO NOTHING;
    """
)


year_table_insert = (
    """
    INSERT INTO Year (Year) 
    VALUES (%s)
    ON CONFLICT (Year) DO NOTHING;
    """
)


fact_table_insert = (
    """
    INSERT INTO Fact_Table (County_ID, School_Type_ID, Employment_Body_ID, Year_ID, No_of_Teachers)
    VALUES (%s, %s, %s, %s, %s);
    """
)



In [300]:
TeacherStatus.columns

Index(['County', 'School Type', 'Employment Body', 'No of Teachers',
       'County (centroid)', 'Year'],
      dtype='object')

In [289]:
# Insert data into the county table
for index,row in TeacherStatus.iterrows():
    try:
        cur.execute(county_table_insert, (row['County'], row['County (centroid)'])) # pass row values as a tuple
        conn.commit()
    except psycopg2.Error as e:
        print(f"Error inserting {row['County']} with centroid {row['County (centroid)']}: {e}")
        conn.rollback()

In [290]:
# Insert data into School_Type table
for school_type in TeacherStatus['School Type'].unique():
    try:
        # Ensure school_type is passed as a tuple with one element
        cur.execute(school_type_insert, (school_type,))
        conn.commit()
    except Exception as e:
        print(f"Error inserting {school_type}: {e}")
        conn.rollback()
        
    

In [291]:
# Insert data in Employment Body_Type
for emp_body in TeacherStatus['Employment Body'].unique():
    try:
        cur.execute(employment_body_insert, (emp_body,))
        conn.commit()
    except psycopg2.Error as e:
        print(f'Error inserting {emp_body}: {e}')
        conn.rollback()

In [292]:
# Insert data into Year table
for year in TeacherStatus['Year'].unique():
    try:
        cur.execute(year_table_insert, (year,))
        conn.commit()
    except psycopg2.Error as e:
        print(f'Error inserting {year}: {e}')
        conn.rollback()

In [295]:
# Function to get IDs from dimension tables
def get_ids(table_name, column_name, value):
    cur.execute(f"SELECT {table_name}_ID FROM {table_name} WHERE {column_name} = %s;", (value,))
    result = cur.fetchone()
    return result[0] if result else None

# Function to get Year_ID based on the Year
def get_year_id(year_value):
    cur.execute("SELECT Year_ID FROM Year WHERE EXTRACT(YEAR FROM Year) = %s;", (year_value.year,))
    result = cur.fetchone()
    return result[0] if result else None

In [313]:
# Insert data into the Fact Table
for index, row in TeacherStatus.iterrows():
    try:
        # Get IDs for each dimension
        county_id = get_ids('County', 'County', row['County'])
        school_type_id = get_ids('School_Type', 'School_Type', row['School Type'])
        emp_body_id = get_ids('Employment_Body', 'Employment_Body', row['Employment Body'])
        year_id = get_year_id(row['Year'])
        
        # Insert data into Fact Table
        cur.execute(fact_table_insert,
                    (county_id, school_type_id, emp_body_id, year_id, row['No of Teachers'])) 
        conn.commit()
    except Exception as e:
        print(f"Error inserting data for {row['County']}: {e}")
        conn.rollback()


In [316]:
# Close connection
cur.close()
conn.close()
